prepare the “Telco-customer-churn” dataset as the following:
- Use Pandas to read the WA_Fn-UseC_-Telco-Customer-Churn CSV dataset
- Kindly set index of the Pandas dataframe by value of CustomerID
- Kindly remove duplicates from the dataframe
- You may need to convert type of "TotalCharges" column from object to
    numeric, one of the possible suggestions is to do as the following:
        - df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        - Take care that the above type conversion will leave some missing values for non-numeric values
        - Why we did this?!, you can try one-hot encoding to know
- Kindly replace the missing values in "TotalCharges" by the average of this column
- Kindly do one-hot encoding for all the categorical columns, except "Contract" and "Churn" columns
- Kindly analyze "Contract" column, and do ordinal encoding for its values as the following:
        - Replace "Month-to-month" by 1
        - Replace "One year" by 2
        - Replace "Two year" by 3
- Kindly convert type of "Contract" column to numeric as we did before, and please don't forget to make sure that there's no missing values left in this column after conversion
- Kindly do label encoding for "Churn" column:
        - Replace "No" by 0
        - Replace "Yes" by 1
- Kindly convert type of "Churn" column to numeric as we did before, and please don't forget to make sure that there's no missing values left in this column after conversion
- Create a separate dataframe contains the “Churn” values, to act as Targets/Labels while training the machine learning model
        - Take care this new dataframe must has "CustomerID" as it's index also
        - kindly remove "Churn" from the original dataframe
- Kindly do normalization for all numerical columns
- Kindly save the two prepared dataframes as CSV files

##### https://stackoverflow.com/questions/24109779/running-get-dummies-on-several-dataframe-columns

In [44]:
# - Use Pandas to read the WA_Fn-UseC_-Telco-Customer-Churn CSV dataset
import pandas as pd

df = pd.read_csv('../datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv', sep=',', index_col=0)

df.drop_duplicates(inplace=True)

# ValueError: Unable to parse string " " at position 488
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace=True)

# please remeber how pd.concat needs a return object
# pd.get_dummies(data = df, columns=['gender','Partner','Dependents', 'PhoneService', 'MultipleLines','OnlineSecurity', 'OnlineBackup','DeviceProtection', 'TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod']).info()
# multiple columns originated here, since pandas 0.19, returned rest of the columns
# df = pd.concat([df, pd.get_dummies(df, columns=['gender','Partner','Dependents', 'PhoneService', 'MultipleLines','OnlineSecurity', 'OnlineBackup','DeviceProtection', 'TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod'])], axis=1)
df = pd.get_dummies(df, columns=['gender','Partner','Dependents', 'PhoneService', 'MultipleLines','OnlineSecurity', 'OnlineBackup','DeviceProtection', 'TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod'])

# there must be a better way to do this, like indexing or something, but :/
# no need for that on pandas 0.19
# del df['gender']
# del df['Partner']
# del df['Dependents']
# del df['PhoneService']
# del df['MultipleLines']
# del df['OnlineSecurity']
# del df['OnlineBackup']
# del df['DeviceProtection']
# del df['TechSupport']
# del df['StreamingTV']
# del df['StreamingMovies']
# del df['PaperlessBilling']
# del df['PaymentMethod']

# Q1: what if there was a null, the conversion to float would be NaN, and then a fillna would be needed after *sigh*
# Q2: what if there was a string left after the replace, the conversion won't occur? (YES) (But it's the dev duty to check)
# Q3: Isn't it better to filla before the conversion? ( #TODO ) (since replace if done correct it should negtate the need for a middle fillna)
# Q4: won't a null be present in the .info() and provid an insight while making the replace, and negating the need for all fillna? ( #TODO )

# Contract didn't have any missing values according to .info() so it's green light here!
Contract_order_dict = {'Month-to-month': 1, 'One year': 2, 'Two year': 3}
df['Contract'] = df['Contract'].replace(Contract_order_dict) # probably will ignore what is not in the dict, and thus need a fillna before turn to numeric(or it will give an error)
df['Contract'] = pd.to_numeric(df['Contract'])
df['Contract'].fillna(df['Contract'].mean(), inplace=True) # fillna with mean of the column, but not required...

Churn_order_dict = {'No': 0, 'Yes': 1}
df['Churn'] = df['Churn'].replace(Churn_order_dict)
df['Churn'] = pd.to_numeric(df['Churn']) # no need to fillna, since it's already done in the previous step (replace) (if not all was included an error might occur btw while conversion)
df['Churn'].fillna(df['Churn'].mean(), inplace=True) #since it's asked to fillna, it's not needed to do it here

df2 = df[['Churn']]
# df2.set_index('CustomerID', inplace=True) # this is not needed, since it's already done in the previous step
df.drop(['Churn'], axis=1, inplace=True)

# Q5: vales of Churn differs depending on when the norm was done (before or after turning into new df), and thus does it differ to the model? ( #TODO )
# Deprecation warning alternative #TODO
df = (df - df.mean()) / (df.max() - df.min())
df2 = (df2 - df2.mean()) / (df2.max() - df2.min())

df.to_csv('../datasets/WA_Fn-UseC_-Telco-Customer-Churn_prepared_1.csv', sep=',', index=False)
df2.to_csv('../datasets/WA_Fn-UseC_-Telco-Customer-Churn_prepared_Churn_2.csv', sep=',', index=False)



C:\Users\ReAegis\AppData\Local\Temp/ipykernel_12008/3107245484.py:57: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = (df - df.mean()) / (df.max() - df.min())
C:\Users\ReAegis\AppData\Local\Temp/ipykernel_12008/3107245484.py:57: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = (df - df.mean()) / (df.max() - df.min())
C:\Users\ReAegis\AppData\Local\Temp/ipykernel_12008/3107245484.py:57: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = (df - df.mean()) / (df.max() - df.min())
